In [76]:
#import modules

import numpy as np
import pandas as pd
from sklearn import datasets as ds
import math as m

In [77]:
#load iris dataset 

iris = ds.load_iris()

In [78]:
#made dictionary for feature names and class names
#f_dic is dictionary for storing names of features according their column index
#t_dic is dictionary for storing class names against integer values
#these will be used later in the code 

feature_names = iris.feature_names
f_dic = {0 : 'sepal length (cm)', 1 : 'sepal width (cm)', 2 : 'petal length (cm)', 3 : 'petal width (cm)'}
target_names = iris.target_names
t_dic = {0 : 'setosa', 1 : 'versicolor', 2 : 'virginica'}

In [79]:
def print_decision_tree(x, features, visited, level) :
    
    #last column(with index 4) is y, we stored that in variable y 
    y = x[:, 4]
    
    
    #check if it is pure node
    unique = np.unique(y)
    lenOfUnique = len(unique)
    if(lenOfUnique == 1) :
        #pure node
        #print Necessary Output
        printLeafNodeData(y, level)
        return
    
    
    
    #check if features array is empty
    count_features_remaining = 0
    for b in visited :
        if(b == False) :
            count_features_remaining += 1
    if(count_features_remaining == 0) :
        #no features left to split
        #print Necessary Output
        printNoFeatureToSplitData(y, level)
        return
    
    
    
    max_info_gain = 0
    final_feature = 0
    final_split_value = 0
    for f in features :
        if(visited[f] == False) :
            temp_gain, temp_feature, temp_split_val = gain(x, f)
            if(temp_gain > max_info_gain) :
                max_info_gain = temp_gain
                final_feature = temp_feature
                final_split_value = temp_split_val
    
    #print Necessary Output
    printSplittingData(y, final_feature, max_info_gain, entropy(x), final_split_value, level)
    
    #split x for left recursive call and right recursive call
    x_left_split = x[x[:, final_feature] <= final_split_value, :]
    x_right_split = x[x[:, final_feature] > final_split_value, :]
    
    #make recursive call to left and right for printing the left and right tree respectively
    print_decision_tree(x_left_split, features, visited, level+1)
    print_decision_tree(x_right_split, features, visited, level+1)
        

In [80]:
def gain(x, f) :
    current_entropy = entropy(x)
    #here we are storing possible values of feature f in poss_vals_feat_f variable
    poss_vals_feat_f = sorted(set(x[:, f]))
    size = len(x)
    max_gain = 0
    final_split_val = 0
    count = len(poss_vals_feat_f)
    i = 0
    while(i < count-1) :
        val1 = poss_vals_feat_f[i]
        val2 = poss_vals_feat_f[i+1]
        #taking mid points of two cosecutive values of poss_vals_feat_f and splitting around that value
        #and maintaining max_gain and final_split_value accordingly
        split_val = (val1+val2)/2
        #split x with columns f values less than and equal to split_val
        x_left_split = x[x[:, f] <= split_val, :]
        #split x with columns f greater than split_val
        x_right_split = x[x[:, f] > split_val, :]
        #here we are calculating entropy for left and right split
        entropy_left = entropy(x_left_split)
        entropy_right = entropy(x_right_split)
        #here we are taking weighted left and right entropy as final entropy after split
        size_left_split = len(x_left_split)
        size_right_split = len(x_right_split)
        temp_entropy = ((size_left_split/size) * entropy_left) + ((size_right_split/size) * entropy_right)
        #here we are calculating info gain after split
        temp_gain = current_entropy - temp_entropy
        #if info gain is greater than max_gain then we are updating our max_gain and final_split_val accordingly
        if(temp_gain > max_gain) :
            max_gain = temp_gain
            final_split_val = split_val
        i += 1
    return max_gain, f, final_split_val

In [81]:
def entropy(x) :
    y = x[:, 4]
    size = len(x)
    unique_classes = set(y)
    count_unique_classes = len(unique_classes)
    #if only class is present then entropy is 0 and we can simply return it
    if(count_unique_classes == 1) :
        return 0
    unique, frequency = np.unique(y, return_counts = True)
    entropy_val = 0
    for f in frequency :
        p = f/size
        entropy_val += -(p * m.log(p, 2))
    return entropy_val

In [82]:
def printLeafNodeData(y, level) :
    unique, freq = np.unique(y, return_counts = True)
    class_pred = unique[0]
    class_name = t_dic[class_pred]
    print()
    print("Level : ", level)
    if(class_pred == 0) :
        print("count of class setosa : ", freq[0])
        print("count of class vergicolor : 0")
        print("count of class virginica : 0")
    elif(class_pred == 1) :
        print("count of class setosa : 0")
        print("count of class vergicolor : ", freq[0])
        print("count of class virginica : 0")
    else :
        print("count of class setosa : 0")
        print("count of class vergicolor : 0")
        print("count of class virginica : ", freq[0])
    print("current entropy : 0.0")
    print("reached leaf node")
    print()

In [83]:
def printNoFeatureToSplitData(y, level) :
    print()
    print("Level : ", level)
    unique, freq = np.unique(y, return_counts = True)
    count_unique = len(unique)
    for i in range(count_unique) :
        class_val = int(unique[i])
        class_val_freq = freq[i]
        print("count of class ", t_dic[class_val], " : ", class_val_freq)
    i = 0
    max_freq = 0
    index_max_freq = 0
    while(i < count_unique) :
        if(freq[i] > max_freq) :
            max_freq = freq[i]
            index_max_freq = i
    print("class predicted : ", t_dic[index_max_freq])
    print("reached leaf node")
    print()

In [84]:
def printSplittingData(y, f, gain, entropy, split_val, level) :
    print()
    print("Level : ", level)
    unique, freq = np.unique(y, return_counts = True)
    count_unique = len(unique)
    if(count_unique == 2 and (0 in unique) == False) :
        print("count of class setosa : 0")
    for i in range(count_unique) :
        class_val = int(unique[i])
        class_val_freq = freq[i]
        print("count of class ", t_dic[class_val], " : ", class_val_freq)
    print("current entropy : ", entropy)
    print("splitting on feature ", f_dic[f], " with gain ratio : ", gain, " around less than and greater than : ", split_val)
    print()

In [85]:
x = iris.data
y = iris.target
features = [0, 1, 2, 3]
splitted = [False, False, False, False]
#here we appended y to x so that when splitting x for left and right recusrive call then we don't have to split y separately
y = y.reshape(-1, 1)
x_mod = np.append(x, y, axis = 1)
print_decision_tree(x_mod, features, splitted, 0)


Level :  0
count of class  setosa  :  50
count of class  versicolor  :  50
count of class  virginica  :  50
current entropy :  1.584962500721156
splitting on feature  petal length (cm)  with gain ratio :  0.9182958340544894  around less than and greater than :  2.45


Level :  1
count of class setosa :  50
count of class vergicolor : 0
count of class virginica : 0
current entropy : 0.0
reached leaf node


Level :  1
count of class setosa : 0
count of class  versicolor  :  50
count of class  virginica  :  50
current entropy :  1.0
splitting on feature  petal width (cm)  with gain ratio :  0.6901603707546748  around less than and greater than :  1.75


Level :  2
count of class setosa : 0
count of class  versicolor  :  49
count of class  virginica  :  5
current entropy :  0.44506485705083865
splitting on feature  petal length (cm)  with gain ratio :  0.21317043093799642  around less than and greater than :  4.95


Level :  3
count of class setosa : 0
count of class  versicolor  :  47
co

In [86]:
class DecisionTree :
    left = None
    right = None
    
    def __init__(self, level, count_class_0, count_class_1, count_class_2, isLeafNode, entropy, split_feature, split_val, gain) :
        self.level = level
        self.count_class_0 = count_class_0
        self.count_class_1 = count_class_1
        self.count_class_2 = count_class_2
        self.isLeafNode = isLeafNode
        self.entropy = entropy
        self.split_feature = split_feature
        self.split_val = split_val
        self.gain = gain

In [87]:
def BuildDecisionTree(x, features, visited, level) :
    y = x[:, 4]
    
    #check if it is pure node
    unique, freq = np.unique(y, return_counts = True)
    lenOfUnique = len(unique)
    if(lenOfUnique == 1) :
        #pure node
        class_val = unique[0]
        class_name = t_dic[class_val]
        class_count_0, class_count_1, class_count_2 = 0, 0, 0
        if(class_val == 0) :
            class_count_0 = freq[0]
        elif(class_val == 1) :
            class_count_1 = freq[0]
        else :
            class_count_2 = freq[0]
        #if current node is leaf node then we are simply make object of current node and are returning it
        root = DecisionTree(level, class_count_0, class_count_1, class_count_2, True, 0, None, None, None)
        return root
    
    
    
    #check if features array is empty
    count_features_remaining = 0
    for b in visited :
        if(b == False) :
            count_features_remaining += 1
    if(count_features_remaining == 0) :
        #no features left to split
        #print Necessary Output
        printNoFeatureToSplitData(y, level)
        return
    
    
    
    max_info_gain = 0
    final_feature = 0
    final_split_value = 0
    for f in features :
        if(visited[f] == False) :
            temp_gain, temp_feature, temp_split_val = gain(x, f)
            if(temp_gain > max_info_gain) :
                max_info_gain = temp_gain
                final_feature = temp_feature
                final_split_value = temp_split_val
    
    #print Necessary Output
    count_unique = len(unique)
    count_class_0 = 0
    if((0 in unique) == False) :
        count_class_0 = 0
    count_class_1, count_class_2 = 0, 0
    for i in range(count_unique) :
        if(unique[i] == 0) :
            class_count_0 = freq[i]
        elif(unique[i] == 1) :
            class_count_1 = freq[i]
        else :
            class_count_2 = freq[i]
    
    
    root = DecisionTree(level, count_class_0, count_class_1, count_class_2, False, entropy(x), final_feature, final_split_value, max_info_gain)    
    
    x_left_split = x[x[:, final_feature] <= final_split_value, :]
    x_right_split = x[x[:, final_feature] > final_split_value, :]
    
    
    #make recursive call to left
    root.left = print_decision_tree(x_left_split, features, visited, level+1)
    root.right = print_decision_tree(x_right_split, features, visited, level+1)
    
    return root

In [88]:
def printDT(root) :
    if(root.isLeafNode == True) :
        print("Level : ", root.level)
        print("count of class setosa : ", root.count_class_0)
        print("count of class versicolor : ", root.count_class_1)
        print("count of class virginica : ", root.count_class_2)
        print("current entropy : 0.0")
        print("reached leaf node")
        print()
        print()
        return
    else :
        print("Level : ", root.level)
        print("count of class setosa : ", root.count_class_0)
        print("count of class versicolor : ", root.count_class_1)
        print("count of class virginica : ", root.count_class_2)
        print("current entropy : ", root.entropy)
        print("splitting on feature ", f_dic[root.split_feature], " with gain ratio : ", root.gain, " around value : ", root.split_val)
        print()
        print()
        if(root.left != None) :
            printDT(root.left)
        if(root.right != None) :
            printDT(root.right)

In [89]:
x_b = iris.data
y_b = iris.target
features_b = [0, 1, 2, 3]
splitted_b = [False, False, False, False]
y_b = y.reshape(-1, 1)
x_mod_b = np.append(x, y, axis = 1)
root = BuildDecisionTree(x_mod, features, splitted, 0)
printDT(root)


Level :  1
count of class setosa :  50
count of class vergicolor : 0
count of class virginica : 0
current entropy : 0.0
reached leaf node


Level :  1
count of class setosa : 0
count of class  versicolor  :  50
count of class  virginica  :  50
current entropy :  1.0
splitting on feature  petal width (cm)  with gain ratio :  0.6901603707546748  around less than and greater than :  1.75


Level :  2
count of class setosa : 0
count of class  versicolor  :  49
count of class  virginica  :  5
current entropy :  0.44506485705083865
splitting on feature  petal length (cm)  with gain ratio :  0.21317043093799642  around less than and greater than :  4.95


Level :  3
count of class setosa : 0
count of class  versicolor  :  47
count of class  virginica  :  1
current entropy :  0.14609425012013633
splitting on feature  petal width (cm)  with gain ratio :  0.14609425012013633  around less than and greater than :  1.65


Level :  4
count of class setosa : 0
count of class vergicolor :  47
count o